# Cleaning and Normalizing Data Mart Files

The files downloaded directly from the [CCC Data Mart](https://datamart.cccco.edu/) are not formatted well. The purpose of this set of scripts is to clean up those files of unhelpful formatting and to normalize for analysis.

For the associated cleaning and denormalization of the EOPS files, see `./ucd_sta_221_project/data_files/cc_eops/clean_eops.ipynb`.

In [136]:
import pandas as pd

In [137]:
top_code = {
    "math": "170100",
    "engl": "150100"
}

In [138]:
path = "ucd_sta_221_project/data_files"
file = f"datamart_math_success_retention_{top_code.get("math")}"

math = pd.read_csv(
    f"{path}/{file}.csv",
    header=None
)
math = math.drop(index=1).reset_index(drop=True) # Contains the word "Transferable"
math.head()

,0,1,2,3,4,5,6,7,8,9,...,231,232,233,234,235,236,237,238,239,240
0,NaN,Fall 2011,Fall 2011,Fall 2011,Fall 2011,Fall 2011,Fall 2012,Fall 2012,Fall 2012,Fall 2012,...,Winter 2022,Winter 2022,Winter 2022,Winter 2022,Winter 2022,Winter 2023,Winter 2023,Winter 2023,Winter 2023,Winter 2023
1,NaN,Enrollment Count,Retention Count,Success Count,Retention Rate,Success Rate,Enrollment Count,Retention Count,Success Count,Retention Rate,...,Enrollment Count,Retention Count,Success Count,Retention Rate,Success Rate,Enrollment Count,Retention Count,Success Count,Retention Rate,Success Rate
2,Alameda,563,441,374,78.33%,66.43%,578,470,379,81.31%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Allan Hancock,965,785,615,81.35%,63.73%,"1,056",833,667,78.88%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,American River,"2,879","2,278","1,826",79.12%,63.42%,"2,683","2,206","1,761",82.22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
new_cols = [f"{a} {b}".strip('_') for a, b in zip(math.iloc[0], math.iloc[1])]
math.columns = new_cols
math.rename(columns={"nan nan": "College"}, inplace=True)
math = math.drop(index=[0, 1]).reset_index(drop=True)  # drop the first two rows

math.head()

,College,Fall 2011 Enrollment Count,Fall 2011 Retention Count,Fall 2011 Success Count,Fall 2011 Retention Rate,Fall 2011 Success Rate,Fall 2012 Enrollment Count,Fall 2012 Retention Count,Fall 2012 Success Count,Fall 2012 Retention Rate,...,Winter 2022 Enrollment Count,Winter 2022 Retention Count,Winter 2022 Success Count,Winter 2022 Retention Rate,Winter 2022 Success Rate,Winter 2023 Enrollment Count,Winter 2023 Retention Count,Winter 2023 Success Count,Winter 2023 Retention Rate,Winter 2023 Success Rate
0,Alameda,563,441,374,78.33%,66.43%,578,470,379,81.31%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allan Hancock,965,785,615,81.35%,63.73%,"1,056",833,667,78.88%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,American River,"2,879","2,278","1,826",79.12%,63.42%,"2,683","2,206","1,761",82.22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Antelope Valley,"1,366","1,117",922,81.77%,67.50%,"1,409","1,178",945,83.61%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bakersfield,"1,144",891,565,77.88%,49.39%,"1,252","1,011",675,80.75%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
# keep only headcounts. We will re-compute rates later
math.columns = math.columns.str.strip()
cols_to_keep = ["College"]
cols_to_keep += [c for c in math.columns if c.endswith("Count")]
math = math.loc[:, cols_to_keep].copy()

math.head()

,College,Fall 2011 Enrollment Count,Fall 2011 Retention Count,Fall 2011 Success Count,Fall 2012 Enrollment Count,Fall 2012 Retention Count,Fall 2012 Success Count,Fall 2013 Enrollment Count,Fall 2013 Retention Count,Fall 2013 Success Count,...,Winter 2020 Success Count,Winter 2021 Enrollment Count,Winter 2021 Retention Count,Winter 2021 Success Count,Winter 2022 Enrollment Count,Winter 2022 Retention Count,Winter 2022 Success Count,Winter 2023 Enrollment Count,Winter 2023 Retention Count,Winter 2023 Success Count
0,Alameda,563,441,374,578,470,379,539,422,307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allan Hancock,965,785,615,"1,056",833,667,"1,101",878,691,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,American River,"2,879","2,278","1,826","2,683","2,206","1,761","2,566","2,055","1,715",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Antelope Valley,"1,366","1,117",922,"1,409","1,178",945,"1,488","1,218",982,...,227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bakersfield,"1,144",891,565,"1,252","1,011",675,"1,360","1,097",695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
cols_to_cast = [c for c in math.columns if c != "College"]

math[cols_to_cast] = math[cols_to_cast].replace({",": ""}, regex=True)

math[cols_to_cast] = (
    math[cols_to_cast]
    .apply(pd.to_numeric, errors="coerce")
    .astype("Int64")
)

math.head()

,College,Fall 2011 Enrollment Count,Fall 2011 Retention Count,Fall 2011 Success Count,Fall 2012 Enrollment Count,Fall 2012 Retention Count,Fall 2012 Success Count,Fall 2013 Enrollment Count,Fall 2013 Retention Count,Fall 2013 Success Count,...,Winter 2020 Success Count,Winter 2021 Enrollment Count,Winter 2021 Retention Count,Winter 2021 Success Count,Winter 2022 Enrollment Count,Winter 2022 Retention Count,Winter 2022 Success Count,Winter 2023 Enrollment Count,Winter 2023 Retention Count,Winter 2023 Success Count
0,Alameda,563,441,374,578,470,379,539,422,307,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,Allan Hancock,965,785,615,1056,833,667,1101,878,691,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,American River,2879,2278,1826,2683,2206,1761,2566,2055,1715,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,Antelope Valley,1366,1117,922,1409,1178,945,1488,1218,982,...,227,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,Bakersfield,1144,891,565,1252,1011,675,1360,1097,695,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [142]:
# Winter intersession terms are very short and not representative;
# drop these columns
math.columns = math.columns.str.strip()
cols_to_drop = [c for c in math.columns if "Winter" in c]
math = math.drop(columns=cols_to_drop)

math.head()

,College,Fall 2011 Enrollment Count,Fall 2011 Retention Count,Fall 2011 Success Count,Fall 2012 Enrollment Count,Fall 2012 Retention Count,Fall 2012 Success Count,Fall 2013 Enrollment Count,Fall 2013 Retention Count,Fall 2013 Success Count,...,Summer 2020 Success Count,Summer 2021 Enrollment Count,Summer 2021 Retention Count,Summer 2021 Success Count,Summer 2022 Enrollment Count,Summer 2022 Retention Count,Summer 2022 Success Count,Summer 2023 Enrollment Count,Summer 2023 Retention Count,Summer 2023 Success Count
0,Alameda,563,441,374,578,470,379,539,422,307,...,434,239,199,172,390,317,294,480,430,397
1,Allan Hancock,965,785,615,1056,833,667,1101,878,691,...,322,290,224,170,282,217,182,339,264,202
2,American River,2879,2278,1826,2683,2206,1761,2566,2055,1715,...,1289,1438,1151,1015,1392,1190,1061,1742,1438,1261
3,Antelope Valley,1366,1117,922,1409,1178,945,1488,1218,982,...,697,822,711,597,748,587,449,880,747,617
4,Bakersfield,1144,891,565,1252,1011,675,1360,1097,695,...,1080,1159,1014,761,1204,1064,818,1137,1001,812


In [143]:
def get_academic_year_terms(year: str):
    """
    Given a year as a string (e.g., "2023"), return a list of academic terms
    for that academic year in the format:
    - Fall {previous_year}
    - Spring {year}
    - Summer {year}

    :param year: The year for which to generate academic terms.
    :return: A list of academic terms for the specified year.
    """
    return [
        f"Fall {int(year) - 1}",
        f"Spring {year}",
        f"Summer {year}"
    ]

get_academic_year_terms("2012")

['Fall 2011', 'Spring 2012', 'Summer 2012']

In [144]:
def build_rates_by_year(year: str, rate: str, df: pd.DataFrame):
    """ 
    Given a reporting year and a rate type (e.g., "Success" or "Retention"),
    computes the rate by summing over each associated term in the reporting year
    and then dividing the appropriate counts.

    :param year: The reporting year as a string (e.g., "2012").
    :param rate: The type of rate to compute ("Success" or "Retention").
    """

    terms = get_academic_year_terms(year)
    if rate == "Success":
        num_cols = [f"{term} Success Count" for term in terms]
        denom_cols = [f"{term} Enrollment Count" for term in terms]
    elif rate == "Retention":
        num_cols = [f"{term} Retention Count" for term in terms]
        denom_cols = [f"{term} Enrollment Count" for term in terms]
    else:
        raise ValueError("Rate must be either 'Success' or 'Retention'.")

    df[f"{year} {rate} Rate"] = (
        df[num_cols].sum(axis=1) / df[denom_cols].sum(axis=1)
    )

    df.drop(columns = num_cols, inplace=True)

    return df

In [145]:
def build_rates(years: list[str], df: pd.DataFrame):
    """
    Given a list of reporting years, computes the Success and Retention rates
    for each combination and updates the DataFrame in place.

    :param years: A list of reporting years as strings (e.g., ["2012", "2013"]).
    :param df: The DataFrame containing the data.
    """
    for year in years:
        for rate in ["Success", "Retention"]:
            df = build_rates_by_year(year, rate, df)
            
    
    # Drop Enrollment Count columns after rates are computed
    df.columns = df.columns.str.strip()
    cols_to_drop = [
        c for c in df.columns if "Enrollment Count" in c
    ]
    df.drop(columns=cols_to_drop, inplace=True)

In [146]:
build_rates(range(2012, 2024), math)

math.head()

,College,2012 Success Rate,2012 Retention Rate,2013 Success Rate,2013 Retention Rate,2014 Success Rate,2014 Retention Rate,2015 Success Rate,2015 Retention Rate,2016 Success Rate,...,2019 Success Rate,2019 Retention Rate,2020 Success Rate,2020 Retention Rate,2021 Success Rate,2021 Retention Rate,2022 Success Rate,2022 Retention Rate,2023 Success Rate,2023 Retention Rate
0,Alameda,0.658505,0.782861,0.687571,0.827119,0.593932,0.783086,0.644478,0.807867,0.628716,...,0.675766,0.839559,0.755714,0.842052,0.644656,0.780803,0.662632,0.781392,0.698471,0.834251
1,Allan Hancock,0.662719,0.814912,0.638959,0.801008,0.628675,0.788159,0.643123,0.807435,0.639649,...,0.573333,0.772403,0.649419,0.816823,0.629732,0.811634,0.51973,0.72947,0.51912,0.762965
2,American River,0.663519,0.806916,0.680633,0.83023,0.693865,0.831361,0.680416,0.82664,0.694554,...,0.667904,0.824101,0.735113,0.841244,0.689805,0.804262,0.68156,0.811592,0.66911,0.809948
3,Antelope Valley,0.671954,0.812906,0.693925,0.841121,0.679663,0.825951,0.65071,0.794322,0.660872,...,0.662455,0.82309,0.664897,0.828358,0.649294,0.837546,0.594568,0.802598,0.607321,0.805324
4,Bakersfield,0.521361,0.77057,0.557572,0.821005,0.556136,0.818211,0.547016,0.81947,0.54734,...,0.623449,0.85701,0.632417,0.838144,0.556286,0.815237,0.554434,0.80855,0.570719,0.836521


In [147]:
# reshape math from wide (one column per year + metric) to long with one row
# per (College, Year)
cols = [c for c in math.columns if c != "College"]

long = math.melt(
    id_vars="College",
    value_vars=cols,
    var_name="year_metric",
    value_name="value"
)

# split the "YYYY Metric" column into Year and Metric
long = long.assign(
    Year=long["year_metric"].str.extract(r"^(\d{4})")[0].astype(int),
    Metric=long["year_metric"].str.replace(r"^\d{4}\s+", "", regex=True).str.strip()
).drop(columns="year_metric")

# pivot so each row is College, Year, Success Rate, Retention Rate
math_yearly = (
    long.pivot_table(
        index=["Year", "College"],
        columns="Metric",
        values="value"
    ).reset_index()
)

math_yearly.columns.name = None
math_yearly = math_yearly.sort_values(["Year", "College"]).reset_index(drop=True)

math_yearly.head()

,Year,College,Retention Rate,Success Rate
0,2012,Alameda,0.782861,0.658505
1,2012,Allan Hancock,0.814912,0.662719
2,2012,American River,0.806916,0.663519
3,2012,Antelope Valley,0.812906,0.671954
4,2012,Bakersfield,0.77057,0.521361


In [148]:
math_yearly.to_csv(f"{path}/{file}_normalized.csv", index=False)